# Template Notebook

In [ ]:
import os
import boto3
from dotenv import load_dotenv
import pandas as pd
from pyathena import connect

pd.options.display.width = 1000
BUCKET = 'ci4cc-hackathon'
load_dotenv()

In [ ]:
s3 = boto3.resource('s3')
# con = connect()


# # %%
# query = """
# SELECT *
# FROM ci4cc_hackathon_database.clinical_onco_patient
# LIMIT 10
# """
# df = pd.read_sql(query, con)
# df

In [ ]:
bucket = s3.Bucket(BUCKET)
for obj in bucket.objects.all():
    print(obj.key, obj.size)